In [1]:
!pip install simpy

In [2]:
from simpy import *
import random
import pandas as pd

In [3]:
def lcg_gen(seed:int, modulus:int, c:int, multiplier:int, num:int):
    randomlist = []
    prevX = seed
    for i in range(num):
        X = (prevX * multiplier + c) % modulus
        randomlist.append(X/modulus)
        prevX = X
    return randomlist

def interarrival_gen(randomlist:list, mean:float):
    import math
    interarrival = []
    for i in range(len(randomlist)):
        t = -1 * mean * math.log(1-randomlist[i])
        interarrival.append(t)
    return interarrival

def servicecustom_gen(randomlist:list):
    servicelist = []
    for i in range(len(randomlist)):
        if randomlist[i] <= 0.24:
            x = 5*randomlist[i]/0.24
            servicelist.append(x)
        elif randomlist[i] <= 1:
            x = (0.52 + randomlist[i])/0.152
            servicelist.append(x)
    return servicelist

In [4]:
seed = 3
modulus = 99
multiplier = 5

maxNumber = 100     # Max number of customers
maxTime = 500.0     # Runtime limit
seed = 12345        # Seed for simulation

In [5]:
def NoInSystem(R):
    return max([0, len(R.put_queue) + len(R.users)])


def AirportSimulation():
    def Customer(env, name, counters, interarrival_time, service_times):
        arrive = env.now
        Qlength = [NoInSystem(counters[i]) for i in range(len(counters))]
        print("%7.4f %s: Here I am. %s" % (env.now, name, Qlength))
        for i in range(len(Qlength)):
            if Qlength[i] == 0 or Qlength[i] == min(Qlength):
                choice = i  # the chosen queue number
                break

        is_priority = random.random() < 0.1
        print('%7.4f %s: Priority Customer: %s' % (env.now, name, is_priority))

        server_status = [1 if counter.count == 1 else 0 for counter in counters]
        event_list.append({'Time': env.now, 'Type': 'Arrival', 'Customer': name, 'Priority': is_priority,
                           'Server1': server_status[0], 'Server2': server_status[1], 'Queue1': Qlength[0], 'Queue2': Qlength[1]})

        with counters[choice].request() as req:
            server_status = ['Busy' if counter.count == 1 else 'Free' for counter in counters]
            print('%7.4f %s: Server Status: %s' % (env.now, name, server_status))

            wait = env.now - arrive
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            yield env.timeout(interarrival_time)

            tib = service_times.pop(0)
            yield env.timeout(tib)

            server_status = [1 if counter.count == 1 else 0 for counter in counters]
            event_list.append({'Time': env.now, 'Type': 'Service End', 'Customer': name, 'Server1': server_status[0],
                               'Server2': server_status[1], 'Priority': is_priority, 'Queue1': Qlength[0], 'Queue2': Qlength[1]})

            print('%7.4f %s: Finished' % (env.now, name))

    print('Airport counter with two servers and one queue')

    random.seed(seed)
    env = Environment()

    counters = [Resource(env), Resource(env)]

    randomlist = lcg_gen(seed, modulus, 0, multiplier, maxNumber*2)
    mean_interarrival = 5

    interarrival_times = interarrival_gen(randomlist, mean_interarrival)
    service_times = servicecustom_gen(randomlist)

    # Initialize an empty list to store events
    event_list = []

    def Source(env, number, counters, interarrival_times, service_times):
        customer_count = 0  # Add a counter variable
        for i in range(number):
            if customer_count >= 100:  # Terminate after 100 customers
                break

            t = interarrival_times.pop(0)
            c = Customer(env, 'Customer%02d' % i, counters, t, service_times)
            env.process(c)
            yield env.timeout(t)
            customer_count += 1

    env.process(Source(env, maxNumber, counters, interarrival_times, service_times))
    env.run(until=maxTime)

    # Convert event_list to a pandas DataFrame
    df = pd.DataFrame(event_list)

    # Display the DataFrame
    #print(df)
    return df

# Call the simulation function
df = AirportSimulation()

Airport counter with two servers and one queue
 0.0000 Customer00: Here I am. [0, 0]
 0.0000 Customer00: Priority Customer: False
 0.0000 Customer00: Server Status: ['Busy', 'Free']
 0.0000 Customer00: Waited  0.000
 3.3165 Customer01: Here I am. [1, 0]
 3.3165 Customer01: Priority Customer: True
 3.3165 Customer01: Server Status: ['Busy', 'Busy']
 3.3165 Customer01: Waited  0.000
 6.0768 Customer02: Here I am. [1, 1]
 6.0768 Customer02: Priority Customer: False
 6.0768 Customer02: Server Status: ['Busy', 'Busy']
 6.0768 Customer02: Waited  0.000
 6.7229 Customer03: Here I am. [2, 1]
 6.7229 Customer03: Priority Customer: False
 6.7229 Customer03: Server Status: ['Busy', 'Busy']
 6.7229 Customer03: Waited  0.000
 9.2481 Customer02: Finished
 9.9273 Customer00: Finished
11.3807 Customer04: Here I am. [0, 2]
11.3807 Customer04: Priority Customer: False
11.3807 Customer04: Server Status: ['Busy', 'Busy']
11.3807 Customer04: Waited  0.000
11.5345 Customer05: Here I am. [1, 2]
11.5345 Custo

In [6]:
df.set_index('Time')#.to_excel('SimLab5AirportSimulation.xlsx')

,Type,Customer,Priority,Server1,Server2,Queue1,Queue2
Time,,,,,,,
0.000000,Arrival,Customer00,False,0,0,0,0
3.316471,Arrival,Customer01,True,1,0,1,0
6.076814,Arrival,Customer02,False,1,1,1,1
6.722873,Arrival,Customer03,False,1,1,2,1
9.248125,Service End,Customer02,False,1,1,1,1
...,...,...,...,...,...,...,...
452.473389,Service End,Customer96,True,1,1,1,1
460.425738,Service End,Customer97,False,1,1,1,0
465.838074,Arrival,Customer99,False,1,0,1,0
